In [17]:
from bili_net import *
# from bilibili_api import errors

In [18]:
## user resp example
UID=176421741
u=user.User(uid=176421741,credential=CRE)
resp=await u.get_followers()
resp


{'list': [{'mid': 488469288,
   'attribute': 0,
   'mtime': 1716178729,
   'tag': None,
   'special': 0,
   'contract_info': {},
   'uname': '弗拉基米尔陀思妥耶夫',
   'face': 'https://i0.hdslb.com/bfs/face/fff11716d2842277371f3bba8f441871ced42586.jpg',
   'sign': '',
   'face_nft': 0,
   'official_verify': {'type': -1, 'desc': ''},
   'vip': {'vipType': 1,
    'vipDueDate': 1664467200000,
    'dueRemark': '',
    'accessStatus': 0,
    'vipStatus': 0,
    'vipStatusWarn': '',
    'themeType': 0,
    'label': {'path': '',
     'text': '',
     'label_theme': '',
     'text_color': '',
     'bg_style': 0,
     'bg_color': '',
     'border_color': ''},
    'avatar_subscript': 0,
    'nickname_color': '',
    'avatar_subscript_url': ''},
   'name_render': {},
   'nft_icon': '',
   'rec_reason': '',
   'track_id': '',
   'follow_time': ''},
  {'mid': 500841415,
   'attribute': 0,
   'mtime': 1715836931,
   'tag': None,
   'special': 0,
   'contract_info': {'is_contractor': True, 'ts': 1715836940},
 

In [19]:

u=user.User(uid=UID,credential=CRE)
resp=await u.get_followings()
resp=await u.get_relation_info()
# resp=await u.get_relation(519200091)
resp

{'mid': 176421741, 'following': 700, 'whisper': 0, 'black': 0, 'follower': 14}

In [20]:
flin_uid,flin_info=await get_following_info([176421741],info_include=['mid','uname']) # example
flin_info

[{'mid': 627998544, 'uname': '大风菌'},
 {'mid': 3546693719034412, 'uname': '快乐圈球'},
 {'mid': 825653, 'uname': '惊面兔在画漫画'},
 {'mid': 3546718433970783, 'uname': '布丁不丁Byte'},
 {'mid': 388262063, 'uname': '目清是个小道士'},
 {'mid': 643873396, 'uname': '平安小财娘'},
 {'mid': 10217431, 'uname': '結城さくな的洋葱港'},
 {'mid': 519200091, 'uname': '游戏王卡片游戏'},
 {'mid': 1297321687, 'uname': '不柒色'},
 {'mid': 646730844, 'uname': '雷军'},
 {'mid': 43904118, 'uname': '网ing少年Joe'},
 {'mid': 670363050, 'uname': '清水芽衣'},
 {'mid': 477533856, 'uname': '九号电动'},
 {'mid': 457464156, 'uname': '重庆IT超市-ROG旗舰店'},
 {'mid': 650589548, 'uname': '金鹰通卡金导'},
 {'mid': 494978109, 'uname': 'LNG九号电竞俱乐部官方'},
 {'mid': 153372667, 'uname': '思卿卿'},
 {'mid': 357121507, 'uname': '柯基小宸'},
 {'mid': 8878641, 'uname': '有颜值的咸鱼君'},
 {'mid': 287795639, 'uname': '短的发布会'},
 {'mid': 8969156, 'uname': '小白测评'},
 {'mid': 3766866, 'uname': '科技美学'},
 {'mid': 68373054, 'uname': '来搞笑的吧'},
 {'mid': 526972283, 'uname': '小望_Official'},
 {'mid': 454152297, 'uname': '神圣泰拉混

In [21]:
from pyvis.network import Network
from typing import Literal,TypedDict

class UserGraph(nx.DiGraph):

    def __init__(self,credential=CRE,incoming_graph_data=None, **attr):
        super().__init__(incoming_graph_data, **attr)
        self.cre=credential

    def _as_pyvis(self) -> Network:
        pv_net=Network(directed=True,notebook=True,cdn_resources='in_line')
        pv_net.from_nx(self)
        return pv_net
    def save_html(self,path='graph.tmp.html'):
        pv_net=self._as_pyvis()
        pv_net.toggle_physics(True)
        pv_net.show_buttons(filter_=['physics'])
        return pv_net.show(path)
    
    def add_relations(self,
                      source,
                      follow_type:Literal['->','<-'],
                      uids=[]
                      ):
        if follow_type=='->':
            edges_to_add = [(source, uid) for uid in uids]
        elif follow_type=='<-':
            edges_to_add = [(uid, source) for uid in uids]
        else:
            raise ValueError
        
        self.add_edges_from(edges_to_add)

    @staticmethod
    def _attr_mapping(attr_lst,key_map):
        mapped_attr_lst=[]
        for old_key,new_key in key_map.items():
            for attr_dct in attr_lst:
                attr_copy=attr_dct.copy()
                if old_key in attr_copy:
                    attr_copy[new_key]=str(attr_copy[old_key])
                    mapped_attr_lst.append(attr_copy)
        return mapped_attr_lst

    def add_user_attr(self,info_with_uid:list[dict]):
        for info_dct in info_with_uid:
            self.add_node(info_dct['mid'],**info_dct)
    
    def info_as_title(self,sep:str='\n',keys=['mid','uname','following','follower']):
        for node,attr in self.nodes(data=True):
            l=[f'{k}={attr[k]}' for k in keys if k in attr]
            title=sep.join(l)
            self.add_node(node,title=title)

    def info_as_label(self,sep:str=',',keys=['uname']):
        for node,attr in self.nodes(data=True):
            l=[str(attr[k]) for k in keys if k in attr]
            label=sep.join(l)
            self.add_node(node,label=label)

    async def _relation_info(self,uid):
        u=user.User(uid,credential=self.cre)
        resp=await u.get_relation_info()
        d={k:v for k,v in resp.items() if k in ('following','follower')}
        return d
    async def update_relation_info(self):
        for node in self.nodes:
            relation_info=await self._relation_info(node)
            self.add_node(node,**relation_info)
        

available attr in pyvis: lable title group(int) size(int)

In [22]:
l=[176421741]
flin_uid,flin_info=await get_following_info(l,info_include=['mid','uname'])
len(flin_info)
# ! 这种情况不能用于直接添加节点，因为没有相应的来源信息
# todo 后续要在类内添加方法，用来获取某个（以及所有）节点的关注信息

50

In [23]:
## add edges
g=UserGraph()
uid=176421741
flin_uid,flin_info=await get_following_info([uid])
g.add_relations(uid,'->',flin_uid)
g.add_user_attr(flin_info)

uid=627998544
flin_uid,flin_info=await get_follower_info([uid])
g.add_relations(uid,'<-',flin_uid)
g.add_user_attr(flin_info)

uid=627998544
flin_uid,flin_info=await get_following_info([uid])
g.add_relations(uid,'->',flin_uid)
g.add_user_attr(flin_info)

# uid=526972283
# flin_uid,flin_info=await get_following_info([uid])
# g.add_relations(uid,'->',flin_uid)
# g.add_user_attr(flin_info)

# uid=388262063
# flin_uid,flin_info=await get_following_info([uid])
# g.add_relations(uid,'->',flin_uid)\
# g.add_user_attr(flin_info)


len(g.nodes)

142

In [25]:
##save
print('nodes num:',len(g.nodes))
g.add_user_attr(flin_info)
await g.update_relation_info()
g.info_as_title()
g.info_as_label()
g.save_html()

nodes num: 142
graph.tmp.html
